# Reading and editing a topography
A topograph/bathymetry is needed to create the domain of interpolation (land-sea mask). The topography/bathymetry is usually extracted from a netCDF file.     
Adapted topographies:
* General Bathymetric Chart of the Oceans ([GEBCO](https://www.gebco.net/))
* [EMODnet Bathymetry](http://www.emodnet-bathymetry.eu)
* Other topographies can be used as long as they define the netCDF variables `lon`, `lat` and `bat` (positive above sea-level and negative below, in `DIVAnd` however, the convention is that `z` is positive in sea water).

This notebook illustrates how the land-sea mask can be modified, for example to exclude a lake.

In [ ]:
using DIVAnd
using PyPlot
if VERSION >= v"0.7.0-beta.0"
    using Dates
    using Statistics
else
    using Compat: @info, @warn, @debug
end
using Compat

The full EMODnet Bathymetry is a quite big file (~1GB) that can be obtained from the bathymetry portal: http://portal.emodnet-bathymetry.eu/?menu=19. In the present notebook we will work with the GEBCO bathymetry with a resolution that has been decreased.

Note that an extremely fine bathymetry is not necessary when working on large domains.

## Data download and domain selection
We create a domain encompassing the north of Corsica.

In [ ]:
dx = dy = 0.125
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

# https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download  gebco_30sec_4.nc
# https://b2drop.eudat.eu/s/x83MSV8yUF934bL/download  gebco_30sec_8.nc
# https://b2drop.eudat.eu/s/o0vinoQutAC7eb0/download  gebco_30sec_16.nc

bathname = "gebco_30sec_8.nc"

if !isfile(bathname)
    @info("Download bathymetry")
    download("https://b2drop.eudat.eu/s/x83MSV8yUF934bL/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

bathisglobal = true;

We have two main tools to load the bathymetry:
1. `DIVAnd.extract_bath` loads the bathymetry at the resolution defined in the netCDF file while 
2. `DIVAnd.load_bath` reduces the resolution to match the resolution of the analysis.

### Extract bathymetry

In [ ]:
?extract_bath

In [ ]:
bx,by,b = extract_bath(bathname,true,lonr,latr)
@show size(b);

In [ ]:
pcolor(bx, by, permutedims(b, [2, 1])); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

### Load bathymetry
With a change of its original resolution

In [ ]:
?load_bath

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr)

pcolor(bx,by,permutedims(b, [2, 1])); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

#@show size(b)

## Mask creation
We start by defining the mask as a 3D matrix containing only `False`, then we set the values to `True` when the actual depth is deeper than the selected values stored in the vector `depthr`.

In [ ]:
depthr = [0,10,20,500]
mask = falses(size(b,1),size(b,2),length(depthr))

for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = b[i,j] >= depthr[k]
        end
    end
end

In [ ]:
pcolor(bx,by,Float64.(permutedims(mask[:,:,1], [2, 1])))
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180)) # fixes the aspect ratio

Another solution is to use the `load_mask` function:

In [ ]:
xmask,ymask,mmask = load_mask(bathname,true,lonr,latr,depthr);

In [ ]:
pcolor(xmask,ymask,Float64.(permutedims(mask[:,:,1], [2, 1]))); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180)) # fixes the aspect ratio

## Mask editing
To edit the mask, you can either modify directly its value if you know the indices, or you can use a selection based on the coordinates.
### Create an artificial island
We simply modify the mask at a given location.

In [ ]:
# Create a copy of the mask for ediding
mask_copy = copy(mask);
mask_copy[63,8,:] .= false;

In [ ]:
pcolor(bx, by, Float64.(permutedims(mask_copy[:,:,1], [2, 1]))); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

### Remove an island (based on indices)

In [ ]:
mask_copy = copy(mask);
mask_copy[48:59,1:10,:] .= true;

In [ ]:
pcolor(bx, by, Float64.(permutedims(mask_copy[:,:,1], [2, 1]))); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(Statistics.mean([ylim()...]) * pi/180))

### Remove an island (based on coordinates). 

This is the recommended approach as you can change resolution easily
`sel_lon` and `sel_lat` are obtained by checking the longitude and latitudes.

In [ ]:
mask_copy = copy(mask);
sel_lon = (bx.>8) .& (bx.<10.);
sel_lat = (by.<43.25);
mask_copy[sel_lon, sel_lat, :] .= true;
pcolor(bx, by, Float64.(permutedims(mask_copy[:,:,1], [2, 1]))); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(Statistics.mean([ylim()...]) * pi/180))

In [ ]:
@show sel_lon;

## Exercice
1. Load the bathymetry around the Black Sea.
2. Create a land sea mask for the following levels: 0, 150, 500 and 1000 meters.
3. Remove the Sea of Azov and the Sea of Marmara.
4. Plot the result.

The solution is available in [06-topography-BlackSea](./solutions/06-topography-BlackSea.ipynb) and should look like this:

<img src="./Images/BlackSea_mask.png" width="250px">